In [24]:
import os
from IPython.display import Markdown
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pydantic import BaseModel
load_dotenv()

True

In [52]:
#Set up the tools that will gona use

search_tool = SerperDevTool()
scrap_tool = ScrapeWebsiteTool()

In [78]:
# For model go to 
MODEL = "groq/mixtral-8x7b-32768"

In [79]:
#Instantiate:

llm = ChatGroq(
    temperature=0.0,
    model_name = MODEL,
    api_key=os.environ["GROQ_API_KEY"])
    



In [84]:
# Corrected Match Researcher Agent
match_researcher = Agent(
    role="Match Researcher",
    goal="Find the match result history of previous matches between {country1} and {country2} in men's soccer. Also consider recent trends and statistics of each country’s performance.",
    backstory="""
    You are a researcher who specializes in finding historical information about previous matches between two countries
    in men's soccer. You consider direct match history as well as the recent performance trends of both teams. Your work
    serves as the foundation for the Match Result Predictor to estimate the outcome of the match.
    """,
    allow_delegation=False,
    llm=llm,
    max_iter=2,
    tools=[search_tool, scrap_tool],
    verbose=True
)

# New Match Predictor Agent
match_predictor = Agent(
    role="Match Predictor",
    goal="Predict the outcome of an upcoming match between {country1} and {country2} based on historical match data and recent performance trends.",
    backstory="""
    You are a data-driven analyst who synthesizes historical match results and recent performance statistics to predict
    the possible outcome of an upcoming match. Your predictions rely on the Match Researcher’s data and statistical analysis.
    Consider win probabilities, scoring trends, and any key player performances.
    """,
    allow_delegation=False,
    llm=llm,
    max_iter=2,
    verbose=True
)


In [85]:
# The task for analyzing match history
analize_match = Task(
    description=(
        "1. Find historical match results between {country1} and {country2} in men's soccer. "
        "This includes analyzing past encounters, the performance of both teams, and relevant statistics. "
        "Also, consider recent trends and team performance leading up to the match."
    ),
    expected_output="A comprehensive list of historical match results, performance statistics, and trend analysis between {country1} and {country2}.",
    agent=match_researcher,
)

# Output format for match prediction results
class OutputFormat(BaseModel):
    country1: str
    country2: str
    prediction: str

# Task for predicting match outcome  
prediction_match_outcome = Task(
    description=(
        "2. Predict the outcome of an upcoming match between {country1} and {country2} based on the historical data "
        "gathered and recent team performance trends."
    ),
    expected_output="A Markdown document with the two countries involved in the match.",
    agent=match_predictor,
    output_format="markdown",  # Define output format as markdown
    output_format_model=OutputFormat,  # Use the OutputFormat model for structured output
    output_format_description=(""" 
        The output format will be a Markdown file with the following structure:
        
        # Match Prediction Report

        ## Match Details
        - **Country 1**: {country1}
        - **Country 2**: {country2}
        
        ---
        *Report generated by Match Predictor Agent.*
    """),
    output_file="mathc_prediction.md"
    
)

In [86]:
crew = Crew(
    agents=[match_researcher,match_predictor],
    tasks=[analize_match,prediction_match_outcome],
    verbose=True
       
        
)

2024-11-14 00:43:32,494 - 140647300317184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [87]:
result = crew.kickoff(inputs={"country1":"chile","country2":"argentina"})

# Agent: Match Researcher
## Task: 1. Find historical match results between chile and argentina in men's soccer. This includes analyzing past encounters, the performance of both teams, and relevant statistics. Also, consider recent trends and team performance leading up to the match.


# Agent: Match Researcher
## Thought: To find the historical match results between Chile and Argentina in men's soccer, I should start by searching the internet for websites that contain this information. I will look for comprehensive match databases or statistics websites that provide the necessary details.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"historical match results chile vs argentina men's soccer\"}"
## Tool Output: 

Search results: Title: Chile national football team: record v Argentina - 11v11
Link: https://www.11v11.com/teams/chile/tab/opposingTeams/opposition/Argentina/
Snippet: 02 Oct 1921, Chile v Argentina, D · 1-1, Unofficial ; 28 Sep 1922, Argentina v Chi